## Filter data result matched_names_tf_idf.xlsx

In [12]:
import pandas as pd
import numpy as np
import time
import re
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/danterangel/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [13]:
filtered = [ 75, 80, 85, 90 ]
#filtered = [90]
datasets_groups = {}

In [14]:
file_quicken = 'full_data_quicken.xlsx'
matched_file = 'matched_names_tf_idf.xlsx'
matched_df = pd.read_excel(matched_file)

full_dataset = pd.read_excel(file_quicken)
dataset = pd.read_excel(file_quicken, usecols=["Payable To"])
dataset = pd.DataFrame({'name': dataset['Payable To'].unique()})
dataset = dataset


In [15]:
print(full_dataset['Payoff Type'].unique())

['Credit Card/Un-secured' 'Insurance' 'Property Tax']


In [16]:
nNames = len(matched_df['Quicken name'].unique())
print("Total dataset", matched_df.shape[0])
print("Total Names found ", nNames)
print("Percent : ", 100 * nNames / dataset.shape[0], '%')

Total dataset 417500
Total Names found  19341
Percent :  91.62442560045479 %


In [17]:
matched_df = matched_df.merge(full_dataset, left_on='Quicken name', right_on='Payable To', how='inner')
matched_df.drop('Payable To', axis='columns', inplace=True)

In [18]:
print("============ Filter dataset ===========")
for i in filtered:
    datasets_groups['matched_m_' + str(i)] = matched_df[matched_df['similarity_token_sort_ratio'] > i]
    print('Dataset with filter > ', i, ' has been saved')
print("============      Done      ===========")

============ Filter dataset ===========
Dataset with filter >  75  has been saved
Dataset with filter >  80  has been saved
Dataset with filter >  85  has been saved
Dataset with filter >  90  has been saved
============      Done      ===========


In [19]:
#datasets_groups

In [21]:
for i in datasets_groups:
    dataset_i = datasets_groups[i].copy()
    data_filtered = pd.DataFrame()
    print("============ Creating dataset " + i + "===========")
    uniques = dataset_i['Quicken name'].unique()
    for j in uniques:
        data_filtered_group = dataset_i[dataset_i['Quicken name'] == j].sort_values('similarity_token_sort_ratio', ascending=False).head(1)
        data_filtered = pd.concat([
            data_filtered, data_filtered_group
        ])
        payoff_type = data_filtered['Payoff Type'].unique()
        sheets = {}
        for z in payoff_type:
            sheet = data_filtered[data_filtered['Payoff Type'] == z].copy()
            sheet.drop('Payoff Type', axis='columns', inplace=True)
            sheets[z] = sheet
    
    with pd.ExcelWriter("gb_matched_names_tf_idf_" + i + ".xlsx") as writer:
        for k in payoff_type:
            sheets[k].to_excel(writer, sheet_name=k.replace('/', ' '),index=False)
    
    print("============       Done " + i + "     ===========")

============ Creating dataset matched_m_75===========
============       Done matched_m_75     ===========
============ Creating dataset matched_m_80===========
============       Done matched_m_80     ===========
============ Creating dataset matched_m_85===========
============       Done matched_m_85     ===========
============ Creating dataset matched_m_90===========
============       Done matched_m_90     ===========
